# Survival Analysis Lab: From Classical to Machine Learning Methods in Heart Attack Patients

In this lab, you'll work with real-world survival data from the **Worcester Heart Attack Study (WHAS500)** — a cohort of 500 patients hospitalised with acute myocardial infarction.

It's good practice to read the official documentation before working with any dataset. For WHAS500, refer to:  
🔗 https://scikit-survival.readthedocs.io/en/stable/api/generated/sksurv.datasets.load_whas500.html

---

## 🎯 What You'll Learn

This lab brings the theory of survival analysis to life. You'll apply key survival analysis methods to a clinical dataset and reflect on their strengths, limitations, and interpretations.

We will walk through:

### 🧭 Lab Sections

**1. Load and Inspect Data**  
Get familiar with the WHAS500 dataset and prepare variables for modelling.

**2. Kaplan-Meier Estimation**  
Estimate and visualise survival curves; compare survival across subgroups.

**3. Cox Proportional Hazards Model**  
Fit a classical Cox model, interpret hazard ratios, and visualise adjusted curves.

**4. Evaluating Performance: The Concordance Index**  
Quantify model discrimination ability using the concordance index (C-index).

**5. Random Survival Forests**  
Model nonlinear survival patterns and visualise predicted survival curves by group.

**6. DeepSurv: Deep Learning for Survival Analysis**  
Use a neural network to learn complex risk relationships and compare model performance.

**7. Summary and Reflections**  
Compare all models, interpret results in context, and reflect on trade-offs between interpretability and flexibility.

---

This lab is designed to complement the associated chapter from my upcoming book, reinforcing survival concepts with realistic, hands-on modelling in a healthcare context.

In [ ]:
%reset

In [ ]:
# !pip install scikit-survival lifelines pandas matplotlib seaborn --quiet
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sksurv.datasets import load_whas500
from sksurv.util import Surv
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.nonparametric import kaplan_meier_estimator

## Section 1: Load and Inspect Data

In [ ]:
# Load dataset
X_struct, y_struct = load_whas500()

# Convert feature array to DataFrame
X = pd.DataFrame(X_struct)

# Convert target array to a DataFrame for inspection 
y_df = pd.DataFrame(y_struct)

# Combine for inspection
X['event'] = y_struct['fstat']
X['time'] = y_struct['lenfol']

# Prepare target for modeling
y = Surv.from_dataframe('event', 'time', X)

X.head()

In [ ]:
from pandasgui import show
gui = show(X)

In [ ]:
# check data types
print(X.dtypes)

### Exploratory Data Analysis

In [ ]:
# Plot a histogram of patient ages to understand their distribution

# Create histogram with 20 bins
X['age'].hist(bins=20)

# Add title and axis labels for clarity
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')

# Display the plot
plt.show()

In [ ]:
# Plot the number of patients by gender

# Map gender codes to labels for readability
gender_labels = {'0': 'Female', '1': 'Male'}
X['gender'].map(gender_labels).value_counts().plot(kind='bar')

# Add a title and rotate x-axis labels for readability
plt.title('Gender Distribution')
plt.xticks(rotation=0)

# Show the plot
plt.show()

### Question: How does survival time vary with congestive heart failure (CHF) status?
💡Use boxplot and summary statistics to compare follow-up time (lenfol) and event rates between patients with and without CHF.

In [ ]:
# Write your code here

In [ ]:

# # 1. View how many patients had congestive heart failure (CHF)
print("CHF value counts:")
print(X['chf'].value_counts())

# # 2. Boxplot: Compare survival time (lenfol / 'time') between CHF groups
plt.figure(figsize=(6, 4))
sns.boxplot(x='chf', y='time', data=X)
plt.title('Survival Time by CHF Status')
plt.xlabel('Congestive Heart Failure (0 = No, 1 = Yes)')
plt.ylabel('Follow-up Time (days)')
plt.show()

# # 3. Compare proportion of deaths by CHF group
death_rates = X.groupby('chf')['event'].mean()
print("\nProportion of deaths in each CHF group:")
print(death_rates)

In [ ]:
# 💡 Uncomment below to reveal the solution

# # 1. View how many patients had congestive heart failure (CHF)
# print("CHF value counts:")
# print(X['chf'].value_counts())

# # 2. Boxplot: Compare survival time (lenfol / 'time') between CHF groups
# plt.figure(figsize=(6, 4))
# sns.boxplot(x='chf', y='time', data=X)
# plt.title('Survival Time by CHF Status')
# plt.xlabel('Congestive Heart Failure (0 = No, 1 = Yes)')
# plt.ylabel('Follow-up Time (days)')
# plt.show()

# # 3. Compare proportion of deaths by CHF group
# death_rates = X.groupby('chf')['event'].mean()
# print("\nProportion of deaths in each CHF group:")
# print(death_rates)

## Section 2: Kaplan-Meier Estimation

In [ ]:
time, survival_prob = kaplan_meier_estimator(X['event'], X['time'])
plt.step(time, survival_prob, where='post')
plt.title('Kaplan-Meier Curve')
plt.xlabel('Time (days)')
plt.ylabel('Survival probability')
plt.grid(True)
plt.show()

In [ ]:
# 💡 Plot overall survival curve
time, survival_prob = kaplan_meier_estimator(y['event'], y['time'])
plt.step(time, survival_prob, where='post', label='Overall')

# 💡 Stratify by CHF status (0 = No CHF, 1 = CHF)
for chf_value in ['0', '1']:
    mask = X['chf'] == chf_value
    time_chf, surv_chf = kaplan_meier_estimator(y['event'][mask], y['time'][mask])
    label = f"CHF = {chf_value}"
    plt.step(time_chf, surv_chf, where="post", label=label)

# Plot formatting
plt.title("Kaplan-Meier Survival Curves by CHF Status")
plt.xlabel("Time (days)")
plt.ylabel("Survival Probability")
plt.legend()
plt.grid(True)
plt.show()


🔧 **Try This**  
Use the plot above to answer the following:

- Which group appears to have the **worst survival**? Why?
- Does the **gap between curves** stay constant over time, or change?
- What might this suggest about the **proportional hazards assumption** for CHF?

*Hint: You could follow this up by fitting a Cox model that includes CHF as a covariate.*


### Question  
Based on the Kaplan-Meier curves stratified by CHF status, which interpretation is most appropriate?

A. Patients with CHF have consistently lower survival probabilities across the follow-up period.  
B. CHF patients initially have worse survival, but then outperform non-CHF patients later.  
C. CHF status only affects survival during the first 30 days.  
D. All patients, regardless of CHF, follow the same survival trajectory.

## Section 3: Cox Proportional Hazards Model

In [ ]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.util import Surv

# Prepare data for Cox model
cox_vars = ['age', 'gender', 'hr', 'chf']
X_model = X[cox_vars]
y_model = Surv.from_dataframe("event", "time", X)

# Fit Cox model
cox_model = CoxPHSurvivalAnalysis()
cox_model.fit(X_model, y_model)

# Display hazard ratios
hazard_ratios = pd.Series(cox_model.coef_, index=cox_vars).apply(np.exp)
hazard_ratios.name = "Hazard Ratio"
display(hazard_ratios)


🔧 **Try This**  
- Which variable has the **strongest association** with survival?
- What does the hazard ratio for **CHF** tell you?
- Is the effect of CHF larger or smaller than age or heart rate?


❓ **Quiz Question**  
Your Cox model shows a hazard ratio of 2.1 for CHF. What does this mean?

A. CHF patients have a 2.1× higher **probability** of dying at any point.  
B. CHF patients have a 2.1× **instantaneous risk** of death, adjusting for other variables.  
C. CHF causes death in 2.1× more patients.  
D. CHF patients survive twice as long as others.


## Section 4: Evaluating Survival Model Performance: The Concordance Index (C-index)

To assess how well a survival model is performing, we need a way to judge not just the predicted survival probabilities, but how well the model **ranks** patients by risk.

The **concordance index (C-index)** is the most widely used metric for this in survival analysis.

### 🧠 What does the C-index measure?

It answers this question:

> "Among all pairs of patients where we know which one died earlier, how often did the model correctly assign a higher risk to the one who died first?"

In simpler terms, it tells us how well the model can **discriminate** between higher-risk and lower-risk patients.

- **C-index = 1.0** → perfect ranking
- **C-index = 0.5** → no better than random guessing
- **C-index < 0.5** → worse than random (usually a red flag)

✅ It works even when not all patients have experienced the event, and it properly handles censored data.

We’ll use it now to evaluate the Cox model, and later apply the same metric to other models like Random Survival Forests and DeepSurv — making comparison easy and fair.


In [ ]:
from sksurv.metrics import concordance_index_censored
# let us first convert the data types of gender and chf so that the functions can handle it (converting from category to integer)
def encode_categoricals(df):
    return df.apply(lambda col: col.cat.codes if col.dtype.name == 'category' else col)
X_model_v2 = encode_categoricals(X_model.copy())
# Get predicted risk scores (higher = higher risk)
risk_scores = cox_model.predict(X_model_v2)

# Evaluate with C-index
c_index, _, _, _, _ = concordance_index_censored(
    y_model['event'], y_model['time'], risk_scores
)

print(f"Concordance Index (Cox model): {c_index:.3f}")

❗ **Note on Risk Score Direction in Cox Models**

When you use `cox_model.predict(X)` from `sksurv`, it returns **log hazard scores** — not survival probabilities.

That means:

- **Higher scores** = **higher risk**
- **Lower scores** = **lower risk**
- So there's **no need to flip the sign** when using these scores for evaluation

In contrast, other models like Random Survival Forests often predict **survival probabilities** — where **lower probabilities** indicate higher risk.  
In those cases, we do apply a minus sign (`-`) to convert survival to risk before calculating the C-index.

✅ Bottom line:  
If you're using Cox model predictions directly, don’t apply a negative sign — they already rank patients by risk in the correct direction.


### Publication-Ready Forest Plots with `lifelines`

While `sksurv` is great for modeling, it doesn’t natively return confidence intervals or standard errors needed for polished visualisations.

To create **publication-ready forest plots**, we’ll use the `lifelines` package. It produces clear, interpretable plots with:
- Hazard ratios on a log scale
- 95% confidence intervals
- A reference line at HR = 1 (no effect)

This is especially helpful when communicating results to non-technical stakeholders or preparing figures for clinical papers.


In [ ]:
from lifelines import CoxPHFitter
import matplotlib.pyplot as plt

# Prepare dataset
df_lifelines = X[['time', 'event', 'age', 'gender', 'hr', 'chf']].copy()

# Fit Cox model
cph = CoxPHFitter()
cph.fit(df_lifelines, duration_col='time', event_col='event')

# Plot with confidence intervals
cph.plot(hazard_ratios=True)
plt.title("Forest Plot: Cox Model Hazard Ratios with 95% CI")
plt.axvline(x=1.0, color='red', linestyle='--')
plt.grid(True)
plt.tight_layout()
plt.show()


## Section 5: Random Survival Forests (RSF)

Random Survival Forests (RSFs) are an extension of random forests for time-to-event data.

They are especially useful when:
- The relationship between predictors and survival is **nonlinear**
- Covariates may **interact** in complex ways
- The **proportional hazards assumption** doesn’t hold

RSFs learn survival patterns by growing many decision trees that split patients based on survival time and censoring status.

Unlike the Cox model, RSFs do not assume a specific form for the hazard — they are fully data-driven and model-free.

Let’s build and train an RSF using our dataset.

In [ ]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv

# Prepare input data
X_model = X[['age', 'gender', 'hr', 'chf']]
y_model = Surv.from_dataframe("event", "time", X)

# Fit Random Survival Forest
rsf = RandomSurvivalForest(n_estimators=100,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=42)
rsf.fit(X_model, y_model)

print("Random Survival Forest model trained.")

### Concordance Index for Random Survival Forests

To fairly compare different survival models, we’ll use the **same evaluation metric** — the concordance index (C-index).

While the Cox model returns log-hazard scores, the RSF predicts **survival functions**.

To compute the C-index for RSF, we:

1. Predict each patient’s **survival probability** at a fixed time point (e.g. 1 year).
2. Convert that to a **risk score**: lower survival = higher risk.
3. Feed those risk scores into the concordance index function.

Let’s calculate it below.


In [ ]:
from sksurv.metrics import concordance_index_censored

# Predict survival function for each patient
surv_funcs_rsf = rsf.predict_survival_function(X_model)

# Get survival probability at 365 days for each patient
surv_1yr = np.array([fn(365) for fn in surv_funcs_rsf])

# Convert to risk scores: lower survival = higher risk
rsf_risk_scores = -surv_1yr

# Compute concordance index
c_index_rsf, _, _, _, _ = concordance_index_censored(
    y_model["event"],
    y_model["time"],
    rsf_risk_scores
)

print(f"Concordance Index (Random Survival Forest): {c_index_rsf:.3f}")

### Feature Importance in RSF

Understanding **which variables influence survival most** is a common goal in both classical and machine learning models.

In `scikit-survival`, the `.feature_importances_` function is often **not implemented** for Random Survival Forests.  
Even if the attribute appears available, trying to use it may raise an error or produce unreliable results.

✅ For reliable results, we recommend using **permutation importance**, which works across models and gives a more robust estimate of which features matter most.

This approach helps us understand:
- Which covariates most affect the model’s survival predictions
- How the RSF prioritises features based on real-world data patterns

🧠 **How Permutation Importance Works**

Permutation importance helps us understand which features a model actually *relies on* to make predictions.

Here’s how it works:

1. **Train the model as usual** on the full dataset.

2. Then, for each feature (e.g. age, heart rate, CHF status):
   - **Randomly shuffle** the values of that feature across all patients.
   - This breaks any relationship between that feature and the outcome.
   - Run the model again on this shuffled version.

3. **Compare performance**:
   - If the model’s accuracy or predictions get *much worse*, the feature was important.
   - If there's *little change*, the model wasn’t using that feature much.

This process is repeated multiple times to get a stable estimate.

✅ It works with **any model**, including Random Survival Forests and deep learning — making it a powerful, model-agnostic way to measure variable importance.

💡 In survival analysis, we can evaluate how shuffling a feature affects predictions like **1-year survival probability** or **concordance index**.

Let’s compute and visualise permutation-based feature importance next.

In [ ]:
#1: Before you shuffle any feature, evaluate the model’s average predicted 1-year survival across all patients:

# Baseline: predicted survival at 1 year for original data
surv_funcs = rsf.predict_survival_function(X_model)
baseline_survival = np.array([fn(365) for fn in surv_funcs])
baseline_score = np.mean(baseline_survival)
print(baseline_score)

In [ ]:
#2: Define a new scoring function that compares to the baseline
# Scoring function: negative average survival probability at 365 days
def rsf_score_absolute_diff(model, X, baseline_score):
    surv_funcs = model.predict_survival_function(X)
    new_surv = np.array([fn(365) for fn in surv_funcs])
    new_score = np.mean(new_surv)
    return abs(new_score - baseline_score)

In [ ]:
# 3. Compute permutation importance manually
#Since this approach doesn’t fit directly into sklearn's permutation_importance, we’ll implement a manual version:

from sklearn.utils import shuffle
import numpy as np
import pandas as pd

# Repeat shuffling to get average importance
n_repeats = 10
importances = []

for col in X_model.columns:
    scores = []
    for _ in range(n_repeats):
        X_temp = X_model.copy()
        X_temp[col] = np.random.permutation(X_temp[col].values)
        score = rsf_score_absolute_diff(rsf, X_temp, baseline_score)
        scores.append(score)
    importances.append(np.mean(scores))

# Convert to series for plotting
importances_series = pd.Series(importances, index=X_model.columns)

In [ ]:
# 4. Plot
importances_series.sort_values().plot.barh()
plt.title("Permutation Importance – Absolute Deviation (1-Year Survival)")
plt.xlabel("Average Change in Predicted Survival")
plt.tight_layout()
plt.show()

### Interpreting Permutation Importance (Absolute Change in Survival)

To understand which features our Random Survival Forest model truly depends on, we use **permutation importance**.

Instead of just checking if survival predictions go down, we check how much they **change**, in any direction.

Here’s what we did:

1. **Computed baseline predictions**:
   - We predicted each patient’s chance of surviving 1 year.
   - Then we averaged those predictions across the whole cohort.

2. **Shuffled each feature** one at a time:
   - This breaks its relationship with the outcome.
   - We recomputed the 1-year survival predictions.

3. **Measured how much the average survival prediction changed**:
   - If a feature is important, shuffling it will lead to a big shift in predictions.
   - If the model wasn’t using the feature, predictions stay about the same.

We repeated this 10 times per feature and averaged the result.

📊 The resulting plot shows how **sensitive** the model’s survival predictions are to each feature — a powerful way to reveal what’s truly driving risk.
### 💡 What makes Permutation Importance so Useful?

- It works with **any kind of model**, not just forests.
- It gives an intuitive sense of **real model reliance**.
- It doesn’t assume anything about the shape or direction of effects.

---

📌 **Bottom Line**:  
Permutation importance tells you **how much the model depends on each variable** by messing with them and seeing what breaks.

It’s simple. Powerful. And great for survival models, especially when built-in importance scores don’t work.


🔧 **Try This: Interpreting the Permutation Plot**

1. Why might **age** have the largest impact on 1-year survival predictions in this model?
2. Are you surprised that **CHF** appears less important than **gender**? What could explain that?
3. Heart rate (hr) seems to have the smallest impact. 
   - Could this be due to its actual clinical role?
   - Or might it reflect something about how this model handles continuous variables?
4. Would you expect these rankings to change if:
   - You used a Cox model instead?
   - You used longer-term survival (e.g. 2-year survival) instead of 1-year?

💡 *Bonus*: Think of a variable not in this dataset that could improve survival prediction. How might you test its importance?


## Section 6: DeepSurv: Deep Learning for Survival Analysis

So far, we’ve explored two classical approaches:
- **Cox proportional hazards**, which assumes a linear log-risk relationship
- **Random Survival Forests**, which model nonlinearities but are tree-based

What if we want to combine the flexibility of nonlinear modelling with the strength of survival frameworks like Cox?

### 🎯 Enter DeepSurv

**DeepSurv** is a deep learning model that generalises the Cox proportional hazards model by replacing the linear predictor with a neural network.

- It **learns complex, nonlinear relationships** between features and survival risk
- Uses the **same partial likelihood loss** as Cox — so it still models **relative risk**
- Supports **censoring**, just like classical survival models
- Can be extended to handle time-varying covariates or competing risks

### ⚙️ How It Works

- Inputs: Patient features (e.g. age, heart rate, comorbidities)
- A feedforward neural network transforms those features
- Output: A **risk score** (just like Cox, but now nonlinear)
- Trained using **negative partial likelihood loss** from Cox

### 📈 Why Use DeepSurv?

- You don’t have to pre-specify interactions or transformations
- Learns directly from data
- Ideal when you suspect complex patterns (e.g. nonlinearity, thresholds, covariate interactions)

We’ll now train a DeepSurv model using the `pycox` library and evaluate it using the same **concordance index** as before — allowing fair comparison with Cox and RSF.


In [ ]:
# STEP 1: Install required packages
#!pip install pycox torchtuples

# STEP 2: Import libraries
import pandas as pd
import numpy as np
import torchtuples as tt
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv
from sklearn.preprocessing import StandardScaler
from sksurv.datasets import load_whas500
from sksurv.util import Surv
from sklearn.model_selection import train_test_split
import torch
import matplotlib.pyplot as plt

# STEP 3: Load dataset
X_raw, y_struct = load_whas500()
X = pd.DataFrame(X_raw)
y_df = pd.DataFrame(y_struct)
y = Surv.from_dataframe("fstat", "lenfol", y_df)

# Prepare features
X_model = X[["age", "gender", "hr", "chf"]].copy()
X_model["gender"] = X_model["gender"].cat.codes
X_model["chf"] = X_model["chf"].cat.codes

# STEP 4: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_model)

# STEP 5: Format target for pycox
y_df = pd.DataFrame.from_records(y)
y_duration = y_df["lenfol"].values
y_event = y_df["fstat"].values

# STEP 6: Train-test split
X_train, X_test, y_train_dur, y_test_dur, y_train_ev, y_test_ev = train_test_split(
    X_scaled, y_duration, y_event, test_size=0.2, random_state=42
)

# STEP 7: Prepare target as tuple of (durations, events)
y_train = (y_train_dur, y_train_ev)
y_test = (y_test_dur, y_test_ev)

# STEP 8: Define and train the DeepSurv model
net = tt.practical.MLPVanilla(
    in_features=X_train.shape[1],
    num_nodes=[32, 32],
    out_features=1,
    activation=torch.nn.ReLU,
    dropout=0.1
)
model = CoxPH(net, tt.optim.Adam)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32') # this ensures that the data and model are using float32
model.fit(X_train, y_train, batch_size=256, epochs=100, verbose=False)
model.compute_baseline_hazards()

In [ ]:
from pycox.evaluation import EvalSurv

# STEP 1: Predict survival functions for the test set
model.compute_baseline_hazards()
surv = model.predict_surv_df(X_test)

# STEP 2: Create an evaluator object
ev = EvalSurv(
    surv,                   # Predicted survival functions (DataFrame: time × patients)
    y_test[0],              # Durations (time to event or censoring)
    y_test[1],              # Events (1 = event, 0 = censored)
    censor_surv='km'        # How to handle censoring (Kaplan-Meier by default)
)

# STEP 3: Calculate concordance index
c_index = ev.concordance_td('antolini')

print(f"Concordance Index (DeepSurv): {c_index:.3f}")

🧪 **Optional: Improve DeepSurv with Early Stopping**

Training deep models on small clinical datasets can lead to overfitting. One way to mitigate this is to use **early stopping**, which monitors validation loss and stops training when performance stops improving.

In `pycox`, you can add early stopping like this:

```python
from torchtuples.callbacks import EarlyStopping

callbacks = [EarlyStopping(patience=10)]

model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=300,
    val_data=(X_test, y_test),
    callbacks=callbacks
)
model.compute_baseline_hazards()


## Section 7: Summary and Reflections

---
### Summary
In this lab, you explored multiple approaches to modelling time-to-event outcomes:

| Model                  | Strengths                                      | Considerations                         |
|------------------------|-----------------------------------------------|----------------------------------------|
| Cox Proportional Hazards | Interpretable, widely used                    | Assumes linear effects, proportional hazards |
| Random Survival Forest  | Handles nonlinearity, robust to outliers      | No hazard ratios; harder to interpret  |
| DeepSurv                | Learns complex patterns from data              | Needs tuning, data hungry              |

You also:
- Calculated **Kaplan-Meier curves** to visualise survival patterns
- Compared models using the **concordance index (C-index)**
- Learned to interpret **survival curves and hazard ratios**
- Visualised patient-level predictions and stratified risk patterns

Each model comes with trade-offs between interpretability, flexibility, and data requirements. There is no single “best” model — the right choice depends on your **data**, **goal**, and **audience**.

---
### Reflect and Apply

1. **Clinical Use Case**
   - Which model would you recommend if interpretability is key?
   - Which would you choose for a purely predictive task?

2. **Data Fit**
   - What assumptions does the Cox model make?
   - When might RSF or DeepSurv be preferable?

3. **Model Limitations**
   - What risks do we face when applying these models blindly?
   - How do censoring and competing risks affect model evaluation?

4. **Visual Thinking**
   - Which survival curves did you find most helpful to interpret?
   - How would you present these results to a clinical or policy audience?

---
### Key Takeaways

- **Survival analysis** is essential for modelling time-to-event outcomes — from hospital readmissions to treatment failure.
- **Interpretability** matters — especially in healthcare. Know what your model tells you, and what it doesn’t.
- **Evaluation** must account for censoring. Concordance index is a great start, but always ask: *“What does this number really mean for decisions?”*
